## Loading & Cleaning SH Clinic Data

In [ ]:
import pandas as pd
import pandas_profiling

%matplotlib inline

### Read in Siloam Data and merge with geocoded addresses. 
Addresses were geocoded by subsets on Year_FirstVisit. (See 'SH Geocoding Addresses.ipynb' notebook)

In [ ]:
year_list = ['2011','2012','2013','2014','2015','2016','2017','2018'] # dropping 2010 and 2019 as they are partial years
sh_df_geocoded = []
    
for year in year_list:
    df_1 = pd.read_csv('data/sh_deid_datapull_20180427.csv')
    df_1['Year_FirstVisit'] = df_1['Year_FirstVisit'].astype(str)

    # subset source df on Year_FirstVisit
    df_1 = df_1.loc[df_1.Year_FirstVisit == year].reset_index(drop=True)

    # read in corresponding df of geocoded address for same year and select only columns of interest
    df_2 = pd.read_csv('data/geocoded_addresses_'+year+'.csv')
    df_2['postcode'] = df_2['postcode'].astype(str)
    df_2 = df_2[['accuracy','formatted_address','postcode','latitude','longitude']]
    
    # left join to df_1 on index
    df_merge = df_1.join(df_2)
    
    # store DataFrame in list
    sh_df_geocoded.append(df_merge)

# concatenate DataFrames in list together
sh_df_geocoded = pd.concat(sh_df_geocoded)

# adding column that shows # of years patient has been coming to clinic
sh_df_geocoded['NumYears'] = sh_df_geocoded['Year_LastVisit'].astype(int) - sh_df_geocoded['Year_FirstVisit'].astype(int)

# select columns on interest
sh_df_geocoded = sh_df_geocoded[['UID', 'Language','Preferred Provider', 'Year_FirstVisit', 'Year_LastVisit','Visit_Count','Active','NumYears', 'accuracy', 'postcode', 'formatted_address', 'latitude',
       'longitude']]

# selecting first 5 digits of postal code column
sh_df_geocoded['postcode'] = sh_df_geocoded.postcode.str[0:5]

In [ ]:
# Haversine formula example in Python
# Author: Wayne Dyck (Adapted code from: https://gist.github.com/rochacbruno/2883505)
# Calculate distances in miles from origin passed in

import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3959 # mi

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return round(d,3)

### Use distance function to compare distances of patient residencies from existing clinic location with distances from potential satellite location. 

In [ ]:
# Add column with distance from SH
sh_location = (36.1187865, -86.778333)
sh_df_geocoded['Dist_from_SH'] = sh_df_geocoded.apply(lambda x: distance(sh_location,(x['latitude'],x['longitude'])), axis=1)

# Add column with distance from potential satellite location
potential_location = (36.0704611,-86.6344346)
sh_df_geocoded['Dist_from_NewSite'] = sh_df_geocoded.apply(lambda x: distance(potential_location,(x['latitude'],x['longitude'])), axis=1)

# remove some additional 'bad' records (did not geocode appropriately)
sh_df_geocoded = sh_df_geocoded.loc[sh_df_geocoded.Dist_from_SH < 300]

In [ ]:
# Cluster distances into buckets

def distance_bucket(value):
    if value <= 2:
        return 'within 2'
    elif value <= 5:
        return 'within 5'
    elif value <= 8:
        return 'within 8'
    elif value <= 10:
        return 'within 10'
    elif value <= 15:
        return 'within 15'
    elif value <= 20:
        return 'within 20'
    else:
        return 'over 20'

In [ ]:
# Apply distance bucket to both SH and New Location (NL) site.
sh_df_geocoded['SH_Dist_Bucket'] = sh_df_geocoded['Dist_from_SH'].apply(lambda x: distance_bucket(x))
sh_df_geocoded['NL_Dist_Bucket'] = sh_df_geocoded['Dist_from_NewSite'].apply(lambda x: distance_bucket(x))

In [ ]:
# Profile Report: Overview of Loaded & Cleaned Data
pandas_profiling.ProfileReport(sh_df_geocoded)